*note, not passing in kid info...model has trained right...so...*
1) get the data to pass through...in one "batch" = pair of frames, for now you need to do comparisons iterating from frame i+1 to i, and run through less if a positive comes up already... - get the label pairs/store the batch info for cost matrix after model - batch lengths will vary, this can be super large for larger raws, so need to come up with some radius function...Jaqaman paper ideas
2) form the data, including the appearances and the centroids to be able to be run with .predict
3)...cost matrix?!?!? for hela, 30 labels is safe, needs to be set somehow...ah the matrices will all be square :)
4) interpret output into cost matrix...the cost matrix needs to be ratio 1:2 (...there is issue rn with the padding for more than number of cells, which will be specific to sets based on if we are able to connect back annotated segments or really how large our testing raws are (compared to annotated...also how will we get the equivalent data with new data...deepcell, celltk...)
5) ...tracking algorithm (relabelling b/w each frame pair using cost matrix...) - compare tracks

In [1]:
from scipy.spatial import distance
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.activations import softmax
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Add, Permute, Input, Concatenate, concatenate
from tensorflow.python.keras.layers import Conv2D, Conv3D, MaxPool2D, AvgPool2D
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.layers import Activation, Softmax
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.regularizers import l2

def siamese_model(input_shape=None, batch_shape=None, reg=1e-5, init='he_normal', permute=False, softmax=True, norm_method='std', filter_size=61):

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    input_1 = Input(shape=input_shape)
    input_2 = Input(shape=input_shape)
    input_3 = Input(shape=(2, ))
    input_4 = Input(shape=(2, ))
    
    # Sequential interface for siamese portion of model
    feature_extractor = Sequential()
    feature_extractor.add(Conv2D(64, (3, 3), kernel_initializer=init, padding='same', kernel_regularizer=l2(reg), input_shape=input_shape))
    feature_extractor.add(BatchNormalization(axis=channel_axis))
    feature_extractor.add(Activation('relu'))
    feature_extractor.add(Conv2D(64, (3, 3), kernel_initializer=init, padding='same', kernel_regularizer=l2(reg)))
    feature_extractor.add(BatchNormalization(axis=channel_axis))
    feature_extractor.add(Activation('relu'))
    feature_extractor.add(MaxPool2D(pool_size=(2, 2)))
    feature_extractor.add(Conv2D(64, (3, 3), kernel_initializer=init, padding='same', kernel_regularizer=l2(reg)))
    feature_extractor.add(BatchNormalization(axis=channel_axis))
    feature_extractor.add(Activation('relu'))
    feature_extractor.add(Conv2D(64, (3, 3), kernel_initializer=init, padding='same', kernel_regularizer=l2(reg)))
    feature_extractor.add(BatchNormalization(axis=channel_axis))
    feature_extractor.add(Activation('relu'))
    feature_extractor.add(MaxPool2D(pool_size=(2, 2)))

    # Create two instances of feature_extractor
    output_1 = feature_extractor(input_1)
    #merged_output1 = concatenate([output_1, input_3], axis=channel_axis)
    ##gmp_1 = GlobalMaxPooling2D()(output1)
    ##gmp_2 = GlobalMaxPooling2D()(input_3)
    ##merged_output1 = concatenate([gmp_1, gmp_2])
    
    output_2 = feature_extractor(input_2)
    #merged_output2 = concatenate([output_2, input_4], axis=channel_axis)

    flat1 = Flatten()(output_1)
    flat2 = Flatten()(output_2)
    #flat3 = Flatten()(input_3)
    #flat4 = Flatten()(input_4)
    merge_1 = concatenate([flat1, input_3])
    merge_2 = concatenate([flat2, input_4])
    # Concatenate outputs from both feature_extractor instances
    """
    merged_outputs = Concatenate(axis=channel_axis)([output_1, output_2])
    flat1 = Flatten()(merged_outputs)
    distance = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(input_3, input_4))))
    print(distance, "     type:", type(distance))
    merge_dist = concatenate([flat1, distance])
    """
    merged_outputs = Concatenate(axis=channel_axis)([merge_1, merge_2])
    # Implement dense net (or call preexisting one?) with the two outputs as inputs
    #dense1 = Dense(128)(merge_dist)
    dense1 = Dense(128)(merged_outputs)
    bn1 = BatchNormalization(axis=channel_axis)(dense1)
    relu1 = Activation('relu')(bn1)
    dense2 = Dense(128)(relu1)
    bn2 = BatchNormalization(axis=channel_axis)(dense2)
    relu2 = Activation('relu')(bn2)
    dense3 = Dense( 3, activation='softmax')(relu2) # changed to 3...

    # Instantiate model
    final_layer = dense3
    model = Model(inputs=[input_1, input_2, input_3, input_4], outputs=final_layer)
    print(model.summary())

    return model

In [2]:
from deepcell.losses import categorical_crossentropy
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from deepcell.utils import rate_scheduler
def train_model_siamese_daughter(model=None, dataset=None, optimizer=None,
                        expt='', it=0, batch_size=1, n_epoch=100,
                        direc_save='/data/models', direc_data='/data/npz_data',
                        lr_sched=rate_scheduler(lr=0.01, decay=0.95),
                        rotation_range=0, flip=True, shear=0, class_weight=None):
    CHANNELS_FIRST = False
    training_data_file_name = os.path.join(direc_data, dataset + '.npz')
    todays_date = datetime.datetime.now().strftime('%Y-%m-%d')

    file_name_save = os.path.join(direc_save, '{}_{}_{}_{}.h5'.format(todays_date, dataset, expt, it))
    file_name_save_loss = os.path.join(direc_save, '{}_{}_{}_{}.npz'.format(todays_date, dataset, expt, it))

    train_dict, (X_test, y_test) = get_data(training_data_file_name, mode='siamese_daughters')

    class_weights = train_dict['class_weights']
    # the data, shuffled and split between train and test sets
    print('X_train shape:', train_dict['X'].shape)
    print('y_train shape:', train_dict['y'].shape)
    print('X_test shape:', X_test.shape)
    print('y_test shape:', y_test[0].shape)
    print('Output Shape:', model.layers[-1].output_shape)

    n_classes = model.layers[-1].output_shape[1 if CHANNELS_FIRST else -1]

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = SiameseDataGenerator(
        rotation_range=rotation_range,  # randomly rotate images by 0 to rotation_range degrees
        shear_range=shear, # randomly shear images in the range (radians , -shear_range to shear_range)
        horizontal_flip=flip,  # randomly flip images
        vertical_flip=flip)  # randomly flip images

    datagen_val = SiameseDataGenerator(
        rotation_range=0,  # randomly rotate images by 0 to rotation_range degrees
        shear_range=0, # randomly shear images in the range (radians , -shear_range to shear_range)
        horizontal_flip=0,  # randomly flip images
        vertical_flip=0)  # randomly flip images

    validation_dict = {'X': X_test, 'y': y_test[0], 'daughters': y_test[1]}

    def count_pairs(y):
        """
        Compute number of training samples needed to (stastically speaking)
        observe all cell pairs.
        Assume that the number of images is encoded in the second dimension.
        Assume that y values are a cell-uniquely-labeled mask.
        Assume that a cell is paired with one of its other frames 50% of the time
        and a frame from another cell 50% of the time.
        """
        # TODO: channels_first axes
        total_pairs = 0
        for image_set in range(y.shape[0]):
            set_cells = 0
            cells_per_image = []
            for image in range(y.shape[1]):
                image_cells = int(y[image_set, image, :, :, :].max())
                set_cells = set_cells + image_cells
                cells_per_image.append(image_cells)

            # Since there are many more possible non-self pairings than there are self pairings,
            # we want to estimate the number of possible non-self pairings and then multiply
            # that number by two, since the odds of getting a non-self pairing are 50%, to
            # find out how many pairs we would need to sample to (statistically speaking)
            # observe all possible cell-frame pairs.
            # We're going to assume that the average cell is present in every frame. This will
            # lead to an underestimate of the number of possible non-self pairings, but it's
            # unclear how significant the underestimate is.
            average_cells_per_frame = int(sum(cells_per_image) / len(cells_per_image))
            non_self_cellframes = (average_cells_per_frame - 1) * len(cells_per_image)
            non_self_pairings = non_self_cellframes * max(cells_per_image)
            cell_pairings = non_self_pairings * 2
            total_pairs = total_pairs + cell_pairings
        return total_pairs

    # This shouldn't remain long term.
    #magic_number = 2048  # A power of 2 chosen just to reduce training time.
    total_train_pairs = count_pairs(train_dict['y'])
    print("total_train_pairs:", total_train_pairs)
    #total_train_pairs = int(total_train_pairs // magic_number)
    #print("total_train_pairs:", total_train_pairs)

    total_test_pairs = count_pairs(y_test[0])
    #print("total_test_pairs:", total_test_pairs)
    #total_test_pairs = int(total_test_pairs // magic_number)
    print("total_test_pairs:", total_test_pairs)
    print("batch size: ", batch_size)
    print("validation_steps: ", total_test_pairs // batch_size)
    test_1, test_2 = datagen.flow(train_dict, batch_size=batch_size, crop_dim=32).next()


    
    # fit the model on the batches generated by datagen.flow()
    loss_history = model.fit_generator(
        datagen.flow(train_dict, batch_size=batch_size, crop_dim=32),
        steps_per_epoch=total_train_pairs // batch_size,
        epochs=n_epoch,
        validation_data=datagen_val.flow(validation_dict, batch_size=batch_size, crop_dim=32),
        validation_steps=total_test_pairs // batch_size,
        callbacks=[
            ModelCheckpoint(file_name_save, monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
            LearningRateScheduler(lr_sched)
        ])

    model.save_weights(file_name_save)
    np.savez(file_name_save_loss, loss_history=loss_history.history)

    return model


In [3]:
from skimage.measure import regionprops
from skimage.transform import resize
import matplotlib.pyplot as plt

def get_inputs(X, y, frames, labels, crop_dim=14, data_format="channels_last"): #labels as a tuple....while loop...
    if data_format == 'channels_first':
        channel_axis = 1
        appearance_shape = (X.shape[channel_axis], len(frames), crop_dim, crop_dim)
    elif data_format == 'channels_last':
        channel_axis = 3
        appearance_shape = (len(frames), crop_dim, crop_dim, X.shape[channel_axis])
    
    #inputs = []
    appearances = np.zeros(appearance_shape, dtype=K.floatx())
    print("appearances: ", appearances.shape)
    centroids = []
    for counter, (frame, cell_label) in enumerate(zip(frames, labels)):
        # Get the bounding box
        y_frame = y[frame] if data_format == 'channels_last' else y[:, frame]
        props = regionprops(np.uint8(y_frame == cell_label))
        minr, minc, maxr, maxc = props[0].bbox
        centroids.append(props[0].centroid)
        
        # Extract images from bounding boxes
        if data_format == 'channels_first':
            appearance = X[:, frame, minr:maxr, minc:maxc]
            resize_shape = (X.shape[channel_axis], crop_dim, crop_dim)
        else:
            appearance = X[frame, minr:maxr, minc:maxc, :]
            resize_shape = (crop_dim, crop_dim, X.shape[channel_axis])

        # Resize images from bounding box
        max_value = np.amax([np.amax(appearance), np.absolute(np.amin(appearance))])
        appearance /= max_value
        appearance = resize(appearance, resize_shape)
        appearance *= max_value
        #appearance.shape = (32, 32, 1)
        #inputs.append(appearance)
        if data_format == 'channels_first':
            appearances[:, counter] = appearance
        else:
            appearances[counter] = appearance
    #inputs.append(appearances)
    #inputs.extend(centroids)
    if data_format == 'channels_first':
        img_shape = (2, X.shape[channel_axis], crop_dim, crop_dim) # passing in pair of images at a time...
    else:
        img_shape = (2, crop_dim, crop_dim, X.shape[channel_axis])
        
    data_shape = (2, 2,) # shape of centroid data

    batch_x_1 = np.zeros(img_shape, dtype=K.floatx())
    batch_x_2 = np.zeros(img_shape, dtype=K.floatx())
    centroid_1 = np.zeros(data_shape, dtype=K.floatx())
    centroid_2 = np.zeros(data_shape, dtype=K.floatx())
    
    batch_x_1[0] = appearances[0]
    batch_x_2[0] = appearances[1]
    
    centroid_1[0] = np.array(centroids[0])
    centroid_2[0] = np.array(centroids[1])

    return [batch_x_1, batch_x_2, centroid_1, centroid_2]


In [4]:
import numpy as np
import os
def run_model_siamese():
    direc_data = '/data/npz_data/cells/HeLa/S3/movie/'
    dataset = 'nuclear_movie_hela1_same'
    #direc_data = 'C:\\Users\\Nora\\Desktop\\'
    #dataset = 'nuclear_movie_hela1_same'
    data = np.load('{}{}.npz'.format(direc_data, dataset))
    segment = 0 #batch
    first_f = 0
    second_f = first_f+1
    labels_in_second = np.uint8(np.unique(data['y'][segment][second_f])) # from annotation
    labels_in_second = np.delete(labels_in_second, np.where(labels_in_second==0))
    labels_in_first = np.uint8(np.unique(data['y'][segment][first_f])) # from annotation
    labels_in_first = np.delete(labels_in_first, np.where(labels_in_first==0))
    
    compairs = []
    for label in labels_in_second:
        for y in labels_in_first:
            compairs.append([y, label]) #while loop with flag to avoid checking for additional positives
    print(compairs)

    """model"""
    model_fn = siamese_model
    model_name = "2018-08-06_nuclear_movie_hela0_same__0.h5"
    MODEL_DIR = "/data/models"
    PREFIX = "cells/HeLa/S3/"
    in_shape = (32, 32, 1)
    weights = os.path.join(MODEL_DIR, PREFIX, model_name)
    #n_features = 3...not used?!
    #window_size = (30, 30)

    run_siamese_model = model_fn(input_shape=in_shape)
    run_siamese_model.load_weights(weights)
    print("weights loaded :)")
    
    model_outputs = []

    for i in range(len(compairs)):
        print("label list: ", compairs[i])
        into_model = get_inputs(data['X'][segment], data['y'][segment], [first_f, second_f], compairs[i], crop_dim = 32)
        
        model_output = run_siamese_model.predict(into_model)
        print("output: ", model_output.shape)
        print(model_output)
        model_outputs.append(model_output)

In [ ]:
run_model_siamese()

[[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [10, 1], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [6, 2], [7, 2], [8, 2], [9, 2], [10, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [6, 3], [7, 3], [8, 3], [9, 3], [10, 3], [1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [6, 4], [7, 4], [8, 4], [9, 4], [10, 4], [1, 5], [2, 5], [3, 5], [4, 5], [5, 5], [6, 5], [7, 5], [8, 5], [9, 5], [10, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [6, 6], [7, 6], [8, 6], [9, 6], [10, 6], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [7, 7], [8, 7], [9, 7], [10, 7], [1, 8], [2, 8], [3, 8], [4, 8], [5, 8], [6, 8], [7, 8], [8, 8], [9, 8], [10, 8], [1, 9], [2, 9], [3, 9], [4, 9], [5, 9], [6, 9], [7, 9], [8, 9], [9, 9], [10, 9], [1, 10], [2, 10], [3, 10], [4, 10], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10]]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to   

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [ ]:
# note, if you scroll down, you can see that the images passed out of the input function are correct,
# but that the model output is the same as the first 

# disregard below this cell

In [4]:
segment = 0 #batch
first_f = 0
second_f = first_f+1
data['X'][segment].shape # segment 0_0

(40, 216, 256, 1)

In [37]:
#data['X'][segment][first_f].shape # frame 0

(216, 256, 1)

In [38]:
#data['X'][segment][second_f].shape # frame 1

(216, 256, 1)

In [5]:
labels_in_second = np.uint8(np.unique(data['y'][segment][second_f])) # from annotation
labels_in_second = np.delete(labels_in_second, np.where(labels_in_second==0))
print(labels_in_second)

[ 1  2  3  4  5  6  7  8  9 10]


In [6]:
labels_in_first = np.uint8(np.unique(data['y'][segment][first_f])) # from annotation
labels_in_first = np.delete(labels_in_first, np.where(labels_in_first==0))
print(labels_in_first)

[ 1  2  3  4  5  6  7  8  9 10]


In [37]:
from skimage.measure import regionprops
from skimage.transform import resize

def get_inputs(X, y, frames, labels, crop_dim=14, data_format="channels_last"): #labels as a tuple....while loop...
    if data_format == 'channels_first':
        channel_axis = 1
        appearance_shape = (X.shape[channel_axis], len(frames), crop_dim, crop_dim)
    elif data_format == 'channels_last':
        channel_axis = 3
        appearance_shape = (len(frames), crop_dim, crop_dim, X.shape[channel_axis])
    
    #inputs = []
    appearances = np.zeros(appearance_shape, dtype=K.floatx())
    print("appearances: ", appearances.shape)
    centroids = []
    for counter, (frame, cell_label) in enumerate(zip(frames, labels)):
        # Get the bounding box
        y_frame = y[frame] if data_format == 'channels_last' else y[:, frame]
        props = regionprops(np.uint8(y_frame == cell_label))
        minr, minc, maxr, maxc = props[0].bbox
        centroids.append(props[0].centroid)
        
        # Extract images from bounding boxes
        if data_format == 'channels_first':
            appearance = X[:, frame, minr:maxr, minc:maxc]
            resize_shape = (X.shape[channel_axis], crop_dim, crop_dim)
        else:
            appearance = X[frame, minr:maxr, minc:maxc, :]
            resize_shape = (crop_dim, crop_dim, X.shape[channel_axis])

        # Resize images from bounding box
        max_value = np.amax([np.amax(appearance), np.absolute(np.amin(appearance))])
        appearance /= max_value
        appearance = resize(appearance, resize_shape)
        appearance *= max_value
        #appearance.shape = (32, 32, 1)
        #plt.imshow(appearance[:,:,0]) # 1, :, 0
        #inputs.append(appearance)
        if data_format == 'channels_first':
            appearances[:, counter] = appearance
        else:
            appearances[counter] = appearance
    #inputs.append(appearances)
    #inputs.extend(centroids)
    if data_format == 'channels_first':
        img_shape = (2, X.shape[channel_axis], crop_dim, crop_dim)
    else:
        img_shape = (2, crop_dim, crop_dim, X.shape[channel_axis])
        
    data_shape = (2, 2,) # shape of centroid data

    batch_x_1 = np.zeros(img_shape, dtype=K.floatx())
    batch_x_2 = np.zeros(img_shape, dtype=K.floatx())
    centroid_1 = np.zeros(data_shape, dtype=K.floatx())
    centroid_2 = np.zeros(data_shape, dtype=K.floatx())
    
    batch_x_1[0] = appearances[0]
    batch_x_2[0] = appearances[1]
    
    centroid_1[0] = np.array(centroids[0])
    centroid_2[0] = np.array(centroids[1])
    print(centroids)
    return [batch_x_1, batch_x_2, centroid_1, centroid_2]


In [29]:
#from deepcell.model_zoo import siamese_model
compairs = []
for label in labels_in_second:
    for y in labels_in_first:
        compairs.append([y, label]) #while loop with flag to avoid checking for additional positives
print(compairs)

"""model"""
model_fn = siamese_model
model_name = "2018-08-06_nuclear_movie_hela0_same__0.h5"
MODEL_DIR = "/data/models"
PREFIX = "cells/HeLa/S3/"
in_shape = (32, 32, 1)
weights = os.path.join(MODEL_DIR, PREFIX, model_name)
#n_features = 3...not used?!
#window_size = (30, 30)

run_siamese_model = model_fn(input_shape=in_shape)
run_siamese_model.load_weights(weights)
print("weights loaded :)")

[[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [10, 1], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [6, 2], [7, 2], [8, 2], [9, 2], [10, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [6, 3], [7, 3], [8, 3], [9, 3], [10, 3], [1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [6, 4], [7, 4], [8, 4], [9, 4], [10, 4], [1, 5], [2, 5], [3, 5], [4, 5], [5, 5], [6, 5], [7, 5], [8, 5], [9, 5], [10, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [6, 6], [7, 6], [8, 6], [9, 6], [10, 6], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [7, 7], [8, 7], [9, 7], [10, 7], [1, 8], [2, 8], [3, 8], [4, 8], [5, 8], [6, 8], [7, 8], [8, 8], [9, 8], [10, 8], [1, 9], [2, 9], [3, 9], [4, 9], [5, 9], [6, 9], [7, 9], [8, 9], [9, 9], [10, 9], [1, 10], [2, 10], [3, 10], [4, 10], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10]]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to   

In [39]:
model_outputs = []

for i in range(len(compairs)):
    print("label list: ", compairs[i])
    into_model = get_inputs(data['X'][segment], data['y'][segment], [first_f, second_f], compairs[i], crop_dim = 32)

    #list_of_nuclear_weights = []
    #for j in xrange(1):
    #nuclear_weights = os.path.join(trained_network_nuclear_directory,  nuclear_prefix + str(j) + ".h5")
    #list_of_nuclear_weights += [nuclear_weights]
    model_output = run_siamese_model.predict(into_model)
    print("output: ", model_output.shape)
    print(model_output)
    model_outputs.append(model_output)

label list:  [1, 1]
appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [2, 1]


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [3, 1]
appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [4, 1]
appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [5, 1]
appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [6, 1]
appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [7, 1]
appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [8, 1]
appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [9, 1]
appearances:  (2, 32

appearances:  (2, 32, 32, 1)
[[0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [1.0352762e-03 9.9855036e-01 4.1435694e-04]]
label list:  [2, 7]


KeyboardInterrupt: 

In [21]:
def run_model_siamese(image, model, win_x=30, win_y=30, split=True):
    channel_axis = 1 if CHANNELS_FIRST else -1
    n_features = model.layers[-1].output_shape[channel_axis]
    

    